In [1]:
from tqdm.notebook import tqdm
import shutil
import utils2p
import yaml
from pathlib import Path
from rye2p.thorimage.settings import FunctionalImagingSettings

# set upstairs or downstairs microscope
microscope = 'downstairs-2p'

# set project directory paths
# PRJ_DATA_DIR = Path("/local/tensor/Haoru/rotation_project")
# RAW_DATA_DIR = Path("/local/tensor/Haoru/rotation_project/raw_data")
# NAS_PROC_DIR = Path("/local/tensor/Haoru/rotation_project/processed_data")

# PRJ_DATA_DIR = Path("/local/tensor/Ruoyi/ruoyi_rotation_project")
# RAW_DATA_DIR = Path("/local/tensor/Ruoyi/ruoyi_rotation_project/raw_data")
# NAS_PROC_DIR = Path("/local/tensor/Ruoyi/ruoyi_rotation_project/processed_data")

PRJ_DATA_DIR = Path("/local/tensor/Ruoyi/ruoyi_mbc1_project")
RAW_DATA_DIR = PRJ_DATA_DIR / "raw_data"
NAS_PROC_DIR = PRJ_DATA_DIR / "processed_data"


# Batch preprocess all ThorImage files from one imaging day

In [2]:
def validate_thorimage_folder(thorimage_folder):
    if not thorimage_folder.joinpath("Experiment.xml").is_file():
        raise FileNotFoundError(f"Experiment.xml file not found.")

    if not thorimage_folder.joinpath("Image_0001_0001.raw").is_file():
        raise FileNotFoundError(f".raw movie file not found.")


def preprocess_thorimage(raw_mov_dir, proc_mov_dir, thorsync_name=None,
                         copy_xml=True,
                         save_settings=True,
                         save_single_tiff=True,
                         save_tiff_sequence=True,
                         frames_per_batch=500):
    """Copy metadata and save movies as .tiff files"""

    validate_thorimage_folder(raw_mov_dir)
    proc_mov_dir.mkdir(parents=True, exist_ok=True)

    save_tiffs = save_single_tiff or save_tiff_sequence

    if copy_xml:
        shutil.copy(raw_mov_dir.joinpath("Experiment.xml"),
                    proc_mov_dir.joinpath("Experiment.xml"))

    if save_settings:
        thorimage_settings = FunctionalImagingSettings.from_xml(
                proc_mov_dir.joinpath("Experiment.xml")
                )
        if thorsync_name is not None:
            thorimage_settings.thorsync_name = thorsync_name

        # Save ThorImage settings to proc_mov_dir
        with open(proc_mov_dir.joinpath('thorimage_settings.yaml'), 'w') as f:
            yaml.dump(thorimage_settings.to_dict(), f, sort_keys=False)

    if save_tiffs:
        print('save tiffs')
        meta = utils2p.Metadata(raw_mov_dir.joinpath("Experiment.xml"))
        raw_file = raw_mov_dir.joinpath("Image_0001_0001.raw")

        stack1, = utils2p.load_raw(raw_file, meta)
        # stack1, stack2 = utils2p.load_raw(raw_file, meta)
        tiff_file = proc_mov_dir.joinpath(raw_file.name).with_suffix('.tif')
        utils2p.save_img(tiff_file, stack1)
        # utils2p.save_img(proc_mov_dir.joinpath("Image_0001_0001_ChanB.tif"), stack2)

        if save_tiff_sequence:
            stk_dir = proc_mov_dir.joinpath('stk')
            stk_dir.mkdir(parents=True, exist_ok=True)

            batch_gen = utils2p.load_stack_batches(tiff_file, frames_per_batch)

            for i, batch in enumerate(batch_gen):
                save_name = stk_dir.joinpath(f"stk_{i:03d}.tif")
                utils2p.save_img(save_name, batch)

    return proc_mov_dir

In [3]:
date_imaged = '2025-10-13'

# get list of all .raw files in the imaging folder
raw_files = sorted(list(RAW_DATA_DIR.joinpath(date_imaged).rglob("Image*.raw")))

# get list of all thorimage folders in the raw data
raw_thorimage_folders = [item.parent for item in raw_files]

for raw_dir in tqdm(raw_thorimage_folders):
    proc_dir = NAS_PROC_DIR.joinpath(raw_dir.relative_to(RAW_DATA_DIR))
    preprocess_thorimage(raw_dir, proc_dir)



  0%|          | 0/2 [00:00<?, ?it/s]

save tiffs
save tiffs
